# Benchmark evaluation

## I. Evaluate algorithm to the benchmark data 

**TODO:** This is not a future-proof way to read in the benchmark data that consists of many files.

Creates a list of Text eobjects with gold and predicted layers

In [1]:
from estnltk import Text, ElementaryBaseSpan, Layer, Span
import ast
from pandas import read_csv
import pandas as pd



def load_data(datafolder='data',description_file='data_description.csv'):
    gold_standard = pd.DataFrame(columns=('text','population'))
    desc = read_csv(description_file)
    for filename,population in zip(desc.file,desc.population):
        data = read_csv(datafolder+'/'+filename)
        texts = data.text
        for text, span in zip(data.text,data.span):
            textobj = Text(text)
            new_layer = Layer('spans',attributes=['labels'])
            textobj.add_layer(new_layer)
            span = ast.literal_eval(span)
            new_span = Span(ElementaryBaseSpan(span['start'],span['end']),new_layer)
            new_span.add_annotation(span)
            textobj.spans.add_span(new_span)
            gold_standard.loc[len(gold_standard)] = {'text':textobj,'population':population}
            
    return gold_standard

In [11]:
gold_standard = load_data()

### I.I. EstNLTK model

In [6]:
from estnltk.taggers import NerTagger
from helper_functions import evaluate_benchmark

nertagger = NerTagger()
results = evaluate_benchmark(gold_standard,nertagger,layer='ner')

100%|████████████████████████████████████████████████████████████████████████████████| 535/535 [00:51<00:00, 10.35it/s]


In [7]:
results['correct'].value_counts()

yes    344
no     191
Name: correct, dtype: int64

### I.II. Kairit model I

**TODO:** Clean up the code. I wanto to see the instantiation of Kairits model and then the call to meodel evaluation. Nothing more!  

In [12]:
from estnltk_neural.taggers.ner.estbertner_tagger import EstBERTNERTagger

kairit_1 = EstBERTNERTagger()

In [13]:
results = evaluate_benchmark(gold_standard,kairit_1,layer='estbertner')

100%|████████████████████████████████████████████████████████████████████████████████| 535/535 [01:09<00:00,  7.72it/s]


In [14]:
results['correct'].value_counts()

yes    345
no     190
Name: correct, dtype: int64

### I.III. Kairit model II

**TODO:** Clean up the code. I wanto to see the instantiation of Kairits model and then the call to meodel evaluation. Nothing more! 

In [16]:
from estnltk import get_resource_paths
kairit_2 = EstBERTNERTagger(output_layer='estbertner2',words_output_layer='nerwords2',model_location=get_resource_paths("estbertner_v2", only_latest=True, download_missing=True))

In [17]:
results = evaluate_benchmark(gold_standard,kairit_2,layer='estbertner2')

100%|████████████████████████████████████████████████████████████████████████████████| 535/535 [01:06<00:00,  8.04it/s]


In [18]:
results['correct'].value_counts()

no     404
yes    131
Name: correct, dtype: int64

## II. Evaluate results

**TODO:** To be cleaned up later!

Use EstNLTK to compute recall together

In [36]:
tbl = read_csv('data_description.csv')

In [37]:
tbl.file = ['recall_sets/koond_1000_levinumad.csv','recall_sets/koond_1000_ulejaanud.csv','recall_sets/koond_1000_mitmetahenduslikud.csv']

In [38]:
tbl[' population'] = ['levinumad','ulejaanud','mitmetahenduslikud']

In [39]:
tbl[' positives'] = [len(levinumad),len(ulejaanud),len(mitmetahenduslikud)]

In [40]:
tbl.to_csv('data_description.csv')

In [4]:
# now fit binomial distributions for each substample

In [42]:
# Fast and dirty version
# Find standard dev for each fraction estimate
# Combine stabndard deviation for the linear combination
# Find CI based on this estimate

# based on:
# https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval#Standard_error_of_a_proportion_estimation_when_using_weighted_data

###levinumad_estnltk
#following is based on corpus_statistics.ipynb
import numpy as np

levinum_weight = 28330.05/ (len(levinumad) * 60629.943)
ulejaanud_weight = 31313.804 / (len(ulejaanud) * 60629.943)
mitmetahenduslik_weight = 986.089 / (len(mitmetahenduslikud) * 60629.943)

weight_vector = [levinum_weight] * len(levinumad) + [ulejaanud_weight] * len(ulejaanud) + [mitmetahenduslik_weight] * len(mitmetahenduslikud)

## EstNLTK
correct_vector_1 = np.concatenate(((levinumad.estnltk_correct=='yes'),(ulejaanud.estnltk_correct=='yes'),(mitmetahenduslikud.estnltk_correct=='yes')))
sample_mean_1 = np.matmul(weight_vector,correct_vector_1)
standard_error_1 = np.sqrt(sample_mean_1 * (1-sample_mean_1) * np.sum(np.square(weight_vector)))
confidence_interval_1 = (sample_mean_1 - standard_error_1 * 1.96, sample_mean_1 + standard_error_1 * 1.96)

### Kairit_1
correct_vector_2 = np.concatenate(((levinumad.Kairit_1_correct=='yes'),(ulejaanud.Kairit_1_correct=='yes'),(mitmetahenduslikud.Kairit_1_correct=='yes')))
sample_mean_2 = np.matmul(weight_vector,correct_vector_2)
standard_error_2 = np.sqrt(sample_mean_2 * (1-sample_mean_2) * np.sum(np.square(weight_vector)))
confidence_interval_2 = (sample_mean_2 - standard_error_2 * 1.96, sample_mean_2 + standard_error_2 * 1.96)

### Kairit_2
correct_vector_3 = np.concatenate(((levinumad.Kairit_2_correct=='yes'),(ulejaanud.Kairit_2_correct=='yes'),(mitmetahenduslikud.Kairit_2_correct=='yes')))
sample_mean_3 = np.matmul(weight_vector,correct_vector_3)
standard_error_3 = np.sqrt(sample_mean_3 * (1-sample_mean_3) * np.sum(np.square(weight_vector)))
confidence_interval_3 = (sample_mean_3 - standard_error_3 * 1.96, sample_mean_3 + standard_error_3 * 1.96)



In [43]:
print(sample_mean_1)
print(confidence_interval_1)

0.5875822776214716
(0.5423782871699572, 0.6327862680729861)


In [44]:
print(sample_mean_2)
print(confidence_interval_2)

0.6025960637963985
(0.5576591752800651, 0.6475329523127319)


In [45]:
print(sample_mean_3)
print(confidence_interval_3)

0.2761996164172543
(0.23514190757483747, 0.31725732525967115)


In [46]:
leaderboard = read_csv('leaderboard.csv')

In [47]:
leaderboard = leaderboard.append({"Algorithm":"EstNLTK"," Recall":sample_mean_1," Recall-95CI% ":confidence_interval_1},ignore_index=True)
leaderboard = leaderboard.append({"Algorithm":"Kairit_I"," Recall":sample_mean_2," Recall-95CI% ":confidence_interval_2},ignore_index=True)
leaderboard = leaderboard.append({"Algorithm":"Kairit_II"," Recall":sample_mean_3," Recall-95CI% ":confidence_interval_3},ignore_index=True)

In [48]:
leaderboard.to_csv('leaderboard.csv')

In [ ]:
# Correct and hard version


# https://stats.stackexchange.com/questions/485266/better-confidence-intervals-for-weighted-average

# find a convolution of distributions 
# https://medium.com/analytics-vidhya/sum-of-two-random-variables-or-the-rocky-path-to-understanding-convolutions-of-probability-b0fc29aca3b5
# https://stackoverflow.com/questions/66218036/question-on-discrete-convolution-with-python
# https://numpy.org/doc/stable/reference/generated/numpy.convolve.html
# https://stackoverflow.com/questions/28901221/faster-convolution-of-probability-density-functions-in-python

In [ ]:
# find 95% confidence inteval for that 